In [8]:
from classify import data_pipeline, eco_selector, reduce, approve, filter_by_conditions
import pandas as pd
import numpy as np
import sys

# Rzepa

In [9]:
corp = 'rzepa'
df = data_pipeline(corp)
df_eco = eco_selector(df)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
rzepa
Configuration finished
Dataset of 14082 samples
0    8468
1    5614
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9971611071682044


100%|██████████| 44/44 [01:24<00:00,  1.92s/it]
